In [179]:
# # Installing necessary dependencies
import sys
# !conda install --yes --prefix {sys.prefix} PIL
# # !{sys.executable} -m pip install ____


# !{sys.executable} -m pip install pytesseract
# !{sys.executable} -m pip install pillow
# !{sys.executable} -m pip install python-poppler
# !{sys.executable} -m pip install opencv-python
# !{sys.executable} -m pip install --upgrade pip
# !conda install -c conda-forge poppler

# !{sys.executable} -m pip install scipy
# !{sys.executable} -m pip install sklearn
# !{sys.executable} -m pip install matplotlib
# !{sys.executable} -m pip install nltk
# !{sys.executable} -m pip install pandas


In [93]:
import os
from pdf2image import convert_from_path
# import pandas as pd
try:
    from PIL import Image
except ImportError:
    import Image
# import numpy as np
import pytesseract
import os

pytesseract.pytesseract.tesseract_cmd = 'C:\\Program Files\\Tesseract-OCR\\tesseract.exe'

In [207]:

# FUnction to extract words from a pdf and save it in as a text file

def pdf_2_txt(path):          
    for file in os.listdir(path): # goes through the folders'
        if file.endswith('.pdf'): # finds pdfs
            p = os.path.join(path,file) # grabs path of pdf
            pdfpages = convert_from_path(p, 500) # Converting each page in the pdf into image file with 500 dpi
            image_counter = 1
            for page in pdfpages:
                filename = f"page_{image_counter}.jpg"
                # Save the image of the page in system
                page.save(filename, 'JPEG')
                image_counter+=1
                text = str(((pytesseract.image_to_string(Image.open(filename))))) # takes image and process into text
                # delete image
                os.remove(filename)
                text = text.replace('-\n', '') # some text processing
                                
                # Checking if the page contains the "Course objective or 
                # description or outline or goals or Learning Outcomes followed by an "end of string" and
                # a new line (As a title)"
                
                if re.match(re.compile("^Course\s(Outline|Objectives|Objective|Description|Goals)$\n"),
                            text) or re.match(re.compile("^Learing Outcomes$\n"), text):
                    filename = filename.replace('.jpg','') # Naming
                    with open(f'{filename}.txt', 'w+') as f: # Opens a text file and writes the characters onto
                        print(text, file=f)
                        print(f'Page_{image_counter}.txt has been saved into {os.getcwd()}')
                    break # stops iteration as soon as page is found               
                
    return #text
                    
                
        


In [205]:
# Implementing pdf to image conversion
path = os.getcwd()
pdf_2_txt(path)
# pages = convert_from_path(path, 500) 
# for page in pages:
#     page.save('test.jpg')

Page_3.txt has been saved into C:\Users\15713\Desktop


In [366]:
# Below, I try to do some text processing and tokenization to see word frequencies

import codecs
import re
import copy
import collections

import numpy as np
import pandas as pd
import nltk
from nltk.stem import PorterStemmer
from nltk.tokenize import WordPunctTokenizer
from __future__ import division
import matplotlib



In [51]:
nltk.download('stopwords') # downloads all stopwords to avoid

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\15713\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [52]:
from nltk.corpus import stopwords 

In [176]:
# In case we have several .txt files of the same pdf, then we use this function to combine into 1 text file (not saved)
def combine_pages(path):
    # Combining pages
    pages = ''
    for file in os.listdir(path):
        if file.endswith('.txt'):
            with codecs.open(file, 'r') as f: 
                page = f.read()
                pages+=page
    return pages

In [180]:
# getting a list of unecessary words like 'the', 'or',... to avoid
avoid = stopwords.words('english')
avoid.append('edu') # adding our own word to the list

In [416]:
nltk.download('punkt') # downloads the tokenizer we need

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\15713\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [567]:
# Function to tokenize a text file into sentences or phrases
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

def get_sentence_counter(path=None, text=None):
    if path:
        path = os.getcwd()
        for file in os.listdir():
            if file.endswith('.txt'):
                with codecs.open(file, 'r') as f: 
                    text = f.read()
    else:
        text = text
    raw_sentences = tokenizer.tokenize(text)
    sentences = []
    for raw_sentence in raw_sentences:
        tmp = raw_sentence.split('\n')
        for x in tmp:
            sentences.append(x)
#         tmp = []
#         break
#     raw_sentences = [x for x in raw_sentences if x != ""]
#     # groups all words and white-spaces, then finds words until a punctuation is reached
#     regex_of_sentence = re.findall('([\w\s]{0,})[^\w\s]', text) 
#     avoid = re.findall('\d', text)
#     avoid.append('')
#     regex_of_sentence = [x for x in regex_of_sentence if x not in avoid]
    # for i in regex_of_sentence:
    #     print(i)
#     np.array(sentences)
    return sentences, len(sentences)

In [506]:
# path = os.getcwd()
# raw_s, s1 = get_sentence_counter(path)
# raw_s

In [204]:
# Function to tokenize a text file into words
def get_text_counter(text, avoid, word_pattern):
#     tokens = WordPunctTokenizer().tokenize(PorterStemmer().stem(text))
    tokens = WordPunctTokenizer().tokenize(text)
    tokens = list(map(lambda x: x.lower(), tokens)) # converting everything to lower-case
    # cleaning up tokens by getting rid of stopwords and finding word characters
    Puncs = [token for token in tokens if re.match(re.compile("\W"), token)] # finding all punctuations
    Nums = [token for token in tokens if re.match(re.compile("\d"), token)] # finding all digit characters
    # adding punctuations and digits to the list of avoided characters/words
    avoid = Puncs + Nums
    tokens = [token for token in tokens if re.match(word_pattern, token) and token not in avoid] 
    # collections.counter() returns the tokens and their frequency as a dictionary ({freq: token})
    return collections.Counter(tokens), len(tokens) 

In [263]:
# Function to make dataframe for both tokenized words and sentences with their abs and relative freqz
def make_df(counter, size, ind_name):
    abs_freq = np.array([x[1] for x in counter]) # the absolute freqz is found from the collections.counter()
    rel_freq = abs_freq/size # relative freqz is calculated by dividing abs_freqz by the size
    index = [x[0] for x in counter] # the tokens are used as an index 
    # creating the dataframe
    df = pd.DataFrame(data=np.array([abs_freq, rel_freq]).T, 
                      index=index,
                      columns=["Absolute frequency","Relative frequency"])
    # Naming the index title
    df.index.name = ind_name
    return df

In [566]:
# # Implementing functions for sentences or phrases
# path = os.getcwd()
# sentences, s2 = get_sentence_counter(path)
# df_sentence = pd.Dataframe({'Sentences': sentences})

In [565]:
# # Implementing functions for sentences or phrases
# path = os.getcwd()
# pages = combine_pages(path) # combining every text file in folder
# word_count, size = get_text_counter(pages, avoid, word_pattern) # returns tokens freqz and count of tokens
# n = 1000 # 1000 of least commonly occuring words
# # least commonly occuring words are chosen with the assumption that a syllabus will only mention
# # technical terms once or twice while other words like 'students' and so on could be redundant
# df_word = make_df(word_count.most_common()[:-n-1:-1], size,
#              ind_name='Least Commonly\n Occuring Words') # finds the least commonly occuring words
# df_word.head(50)

In [548]:
# Working with 'Indeed Dataset'
path = os.path.join(os.getcwd(), 'indeed_dataset.csv')
df = pd.read_csv(path)

In [549]:
df.head(10)

,Title,Company,Location,Links,Salary,Description
0,Senior Software Engineer Security and Authenti...,MathWorks,"Natick, MA 01760",https://indeed.com/viewjob?jk=dedaae73a1bb2555,None,We are looking for a highly motivated work-fro...
1,None,Citizens,"Greenville, RI 02828",https://indeed.com/viewjob?jk=8b0ad10599f3a304,None,Job Description:\nAre you looking for a new an...
2,"2021745 Cyber Security Analysts $215,000.00",B4CORP,"Reston, VA",https://indeed.com/viewjob?jk=407b2677aa6ccc25,"Up to $215,000 a year",US CITIZENSHIP REQUIRED FOR THIS POSITION: Yes...
3,Senior Cyber Security Analyst,TherapyNotes.com,Remote,https://indeed.com/viewjob?jk=0dd5712bac6c57aa,None,"Cyber Security Analyst, Incident Response, Cyb..."
4,Cyber Security Analyst,NDK,"Morrisville, NC•Remote",https://indeed.com/viewjob?jk=2d39ed3a3baf0863,"Up to $130,000 a year",Information Security Engineer\nThe Information...
5,None,CentralSquare Technologies,+1 locationRemote,https://indeed.com/viewjob?jk=56f081f9afeb537e,"$85,000 - $100,000 a year","This role leads initiatives in risk, product s..."
6,Cyber Security Engineer,Softnice Inc.,Remote,https://indeed.com/viewjob?jk=8b05a98ec7ae4b42,None,Job description:\nStrong IT skills and knowled...
7,None,Amazon.com Services LLC,"New York, NY",https://indeed.com/viewjob?jk=5025373d360ec8f3,None,"Bachelor’s degree in Computer Science, Compute..."
8,Cyber Security and Networking Apprentice,United States Navy Career Center South Bay,"Redwood City, CA+2 locations",https://indeed.com/viewjob?jk=77b578b9c0642d23,"$50,000 - $95,000 a year",Your Job\nIndeed is seeking an Associate Appli...
9,None,NDK,"Morrisville, NC 27560•Remote",https://indeed.com/viewjob?jk=ed0c996377d869d3,"Up to $130,000 a year","THIS IS AN ENTRY LEVEL POSITION, NO TRAINING A..."


In [550]:
# Creating a dataframe with title as index and description as an attribute
title, description = df['Title'].to_list(), df['Description'].to_list()
indeed_df = pd.DataFrame(data={"Title": title,
                               "Description": description} 
                         )
# removing rows with 'None' for their Job-Title
indeed_df = indeed_df[indeed_df.index != 'None']

In [551]:
indeed_df.head(10)

,Title,Description
0,Senior Software Engineer Security and Authenti...,We are looking for a highly motivated work-fro...
1,None,Job Description:\nAre you looking for a new an...
2,"2021745 Cyber Security Analysts $215,000.00",US CITIZENSHIP REQUIRED FOR THIS POSITION: Yes...
3,Senior Cyber Security Analyst,"Cyber Security Analyst, Incident Response, Cyb..."
4,Cyber Security Analyst,Information Security Engineer\nThe Information...
5,None,"This role leads initiatives in risk, product s..."
6,Cyber Security Engineer,Job description:\nStrong IT skills and knowled...
7,None,"Bachelor’s degree in Computer Science, Compute..."
8,Cyber Security and Networking Apprentice,Your Job\nIndeed is seeking an Associate Appli...
9,None,"THIS IS AN ENTRY LEVEL POSITION, NO TRAINING A..."


In [428]:
# Converting the above dataframe into dictionary for iteration {title: description}
indeed = {}
for (t, d) in zip(title, description): # loops through the list title and description
    if t in indeed: # if the job title already exists, we lump the descriptions together
        indeed[t]+=d
    else:
        indeed[t] = d # sets the title as key and the description as the value


In [534]:
# iterating through the Job's to tokenize into sentences/phrases
indeed_tokenized_dict = {}
for key in indeed:
    text = indeed[key]
    # Tokenize into sentence or phrase
    sentence_count, s1 = get_sentence_counter(text=text)
    indeed_tokenized_dict[key] = sentence_count


In [564]:
# Putting in dataframe with keys/job titles as titles and the values as the tokenized sentences just for visual purposes
df_concat = pd.DataFrame()
for k,v in indeed_tokenized_dict.items():
    if k != 'None':
        df = pd.DataFrame({k: v})
        df_concat = pd.concat([df_concat, df], axis=1)

df_concat.T.head(10)

,0,1,2,3,4,5,6,7,8,9,...,456,457,458,459,460,461,462,463,464,465
Senior Software Engineer Security and Authentication,We are looking for a highly motivated work-fro...,"Your mission will be to grow our services, man...",Responsibilities will include selling security...,This sales position is a work from home positi...,You will inherit multiple current customer acc...,This is a challenging and rewarding sales oppo...,The ability to move and grow with our company ...,We are a team of overachievers who thrive in a...,eSecurity Solutions leverages its relationship...,Our sales staff provides knowledgeable consult...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"2021745 Cyber Security Analysts $215,000.00",US CITIZENSHIP REQUIRED FOR THIS POSITION: Yes,RELOCATION ASSISTANCE: No relocation assistanc...,CLEARANCE TYPE: Secret,"TRAVEL: Yes, 10% of the Time",Description,Join Northrop Grumman on our continued mission...,Enjoy a culture where your voice is valued and...,We take pride in creating purposeful work and ...,With our competitive pay and comprehensive ben...,Northrop Grumman is seeking Information System...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Senior Cyber Security Analyst,"Cyber Security Analyst, Incident Response, Cyb...",NDK Cyber have partnered with one the world’s ...,We’re recruiting a Cyber Security Analyst to j...,This team are making huge investments and will...,This Cyber Security Analyst role is part of gr...,We’re looking for someone who’s operated in a ...,What you’ll be doing,As a Cyber Security Analyst you’ll have the op...,· Responding to incidents and conducting inves...,· Ensuring the security technology provided by...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Cyber Security Analyst,Information Security Engineer,The Information Security Engineer is responsib...,The engineer implements security measures that...,Provide training to employees on security best...,Responsibilities:,Design computer security strategy and engineer...,"Develop, execute and track the performance of ...","Identify, define and document system security ...","Configure, troubleshoot and maintain security ...",Install software that monitors systems and net...,...,Knowledge of process-focused methodologies for...,Exposure to IT Risk and Process frameworks: CO...,CRISC minimum certification requested or worki...,"JPMorgan Chase & Co., one of the oldest financ...",Our history spans over 200 years and today we ...,We recognize that our people are our strength ...,We are an equal opportunity employer and place...,We do not discriminate on the basis of any pro...,"In accordance with applicable law, we make rea...",Equal Opportunity Employer/Disability/Veterans
Cyber Security Engineer,Job description:,Strong IT skills and knowledge including hardw...,Meticulous attention to detail,Ability to use logic and reasoning to identify...,A forensic approach to challenges,A deep understanding of how hackers work and a...,Ability to seek out vulnerabilities in IT infr...,Job Type: Full-time,Schedule:,8 hour shift,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Cyber Security and Networking Apprentice,Your Job,Indeed is seeking an Associate Application Sec...,You will interact with partners across all tec...,Responsibilities (What You’ll Do),"Work with other teams to identify, resolve, an...",Evaluating or creating new technologies and se...,Perform design reviews and risk assessments fo...,Perform analysis of log files and data outputs.,Perform triage of incoming issues using ticket...,Create and maintain documentation for new and ...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Cyber Threat Intelligence Analyst,HOW TO APPLY,STEP 1: Click on the “Apply on Company Site” b...,STEP 2: Click the “Start” button to begin.,You will be prompted to either Sign In to cont...,STEP 3: Follow the step-by-step process to sub...,You will be guided through each step.,You must complete all sections of the form AND...,1.,"Your resume, specifically noting relevant work...",

In [552]:
# shows the description
indeed['SOC Analyst']

'Summit Technologies Inc. has an immediate need for a Senior Imprivata Security Engineer. In this position you will be a key player in migrating Care Delivery organizations into our clients Identity and Access Management framework. They are building an enterprise-wide implementation of Imprivata. This position is contract to hire and 100% remote work. Our client is not able to sponsor candidates.\n\nResponsibilities:\nYou will participate in the technical planning, buildout, and operations of this enterprise-scale Imprivata implementation\nUse your Imprivata expertise, identity knowledge, and the enterprise technology capability stack to ensure that their integration into Imprivata is successful and secure\nBuild and configure Imprivata appliances\nProvide engineering expertise for build, integration, and expansion operations of the Imprivata platform\nManage operational response and meet response SLAs for support of Imprivata-connected business units\nCollaborate with care delivery su

In [553]:
# shows the sentences in the description tokenized
indeed_tokenized_dict['SOC Analyst']

['Summit Technologies Inc. has an immediate need for a Senior Imprivata Security Engineer.',
 'In this position you will be a key player in migrating Care Delivery organizations into our clients Identity and Access Management framework.',
 'They are building an enterprise-wide implementation of Imprivata.',
 'This position is contract to hire and 100% remote work.',
 'Our client is not able to sponsor candidates.',
 'Responsibilities:',
 'You will participate in the technical planning, buildout, and operations of this enterprise-scale Imprivata implementation',
 'Use your Imprivata expertise, identity knowledge, and the enterprise technology capability stack to ensure that their integration into Imprivata is successful and secure',
 'Build and configure Imprivata appliances',
 'Provide engineering expertise for build, integration, and expansion operations of the Imprivata platform',
 'Manage operational response and meet response SLAs for support of Imprivata-connected business units',

In [ ]:
df